# Modelos Lineales Mixtos en Python

## Índice
1. [Introducción](#intro)
2. [Configuración del Entorno](#setup)
3. [Conceptos Fundamentales](#concepts)
4. [Análisis con Distribución Normal](#normal)
5. [Análisis con Distribución de Poisson](#poisson)
6. [Análisis con Distribución Binomial Negativa](#negative-binomial)
7. [Análisis con Distribución Cero-Inflada](#zero-inflated)
8. [Pruebas Post Hoc y Efectos Marginales](#post-hoc)
9. [Conclusiones](#conclusions)
10. [Referencias](#references)

## 1. Introducción <a id='intro'></a>

Los modelos lineales mixtos son herramientas estadísticas poderosas para analizar datos que presentan estructuras complejas, como medidas repetidas o datos jerárquicos. En este documento, exploraremos cómo implementar y interpretar estos modelos en Python, utilizando diferentes distribuciones de datos: normal, Poisson, binomial negativa y cero-inflada.

## 2. Configuración del Entorno <a id='setup'></a>

Primero, instalamos los paquetes necesarios y importamos las librerías requeridas.

In [10]:
# Instalar las librerías necesarias
!pip install --user statsmodels pandas numpy matplotlib seaborn

# Verificar y agregar la ruta de instalación de paquetes locales al sys.path
import sys
package_path = '/home/docker/.local/lib/python3.8/site-packages'
if package_path not in sys.path:
    sys.path.append(package_path)

# Ahora intentamos importar las librerías
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Librerías importadas correctamente.")



Librerías importadas correctamente.


## 3. Conceptos Fundamentales <a id='concepts'></a>

> **Nota:** Los modelos lineales mixtos (MLM) combinan efectos fijos y aleatorios para capturar tanto las relaciones sistemáticas como la variabilidad no explicada en los datos.

La elección de la distribución adecuada es crucial para modelar correctamente los datos, incluyendo distribuciones normal, Poisson, binomial negativa y cero-inflada.

## 4. Análisis con Distribución Normal <a id='normal'></a>

En este análisis, ajustamos un modelo lineal mixto utilizando la distribución normal para modelar variables continuas. Por ejemplo, podemos estudiar cómo un programa educativo afecta los puntajes de los estudiantes.

In [11]:
# Creación de datos simulados
data_normal = pd.DataFrame({
    'score': np.random.normal(75, 10, 100),
    'program': np.random.choice(['Control', 'Experimental'], size=100),
    'school': np.random.choice(['Escuela_1', 'Escuela_2', 'Escuela_3'], size=100)
})

# Ajuste del modelo lineal mixto
model_normal = smf.mixedlm("score ~ program", data_normal, groups="school")
result_normal = model_normal.fit()

# Resumen del modelo
print(result_normal.summary())

              Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    score    
No. Observations:      100        Method:                REML     
No. Groups:            3          Scale:                 86.8867  
Min. group size:       32         Log-Likelihood:        -361.7286
Max. group size:       35         Converged:             Yes      
Mean group size:       33.3                                       
------------------------------------------------------------------
                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept               75.297    1.257 59.908 0.000 72.834 77.761
program[T.Experimental] -0.617    1.873 -0.329 0.742 -4.289  3.055
school Var               0.000                                    



/home/docker/.local/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/home/docker/.local/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/docker/.local/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


Interpretación: El coeficiente asociado al programa indica si hubo un efecto significativo del programa experimental en comparación con el grupo de control.

## 5. Análisis con Distribución de Poisson <a id='poisson'></a>

La distribución de Poisson se utiliza para modelar conteos de eventos. Aquí se muestra cómo ajustar un modelo utilizando esta distribución para analizar, por ejemplo, el número de defectos en una línea de producción.

In [12]:
# Creación de datos simulados
data_poisson = pd.DataFrame({
    'defects': np.random.poisson(2, 100),
    'hours': np.random.poisson(8, 100),
    'machine': np.random.choice(['Máquina_1', 'Máquina_2', 'Máquina_3'], size=100)
})

# Ajuste del modelo de Poisson
model_poisson = smf.poisson("defects ~ hours", data_poisson).fit()

# Resumen del modelo
print(model_poisson.summary())

Optimization terminated successfully.
         Current function value: 1.700677
         Iterations 4
                          Poisson Regression Results                          
Dep. Variable:                defects   No. Observations:                  100
Model:                        Poisson   Df Residuals:                       98
Method:                           MLE   Df Model:                            1
Date:                Wed, 16 Oct 2024   Pseudo R-squ.:               0.0003170
Time:                        16:30:38   Log-Likelihood:                -170.07
converged:                       True   LL-Null:                       -170.12
Covariance Type:            nonrobust   LLR p-value:                    0.7426
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7124      0.202      3.522      0.000       0.316       1.109
hours          0.0077      0.

Interpretación: El modelo nos permite evaluar cómo las horas de operación afectan el número de defectos observados.

## 6. Análisis con Distribución Binomial Negativa <a id='negative-binomial'></a>

La distribución binomial negativa es útil cuando existe sobredispersión en los datos de conteo. Ajustamos un modelo binomial negativo para mejorar el ajuste respecto a un modelo de Poisson.

In [ ]:
# Ajuste del modelo binomial negativo
import statsmodels.discrete.discrete_model as smd

model_negbin = smd.NegativeBinomial(data_poisson['defects'], sm.add_constant(data_poisson['hours'])).fit()

# Resumen del modelo
print(model_negbin.summary())

Interpretación: Este modelo permite manejar mejor la variabilidad adicional presente en los datos de conteo.

## 7. Análisis con Distribución Cero-Inflada <a id='zero-inflated'></a>

En algunos casos, los datos de conteo contienen un exceso de ceros. La distribución cero-inflada nos permite modelar estos datos de manera más precisa. Por ejemplo, el número de visitas a un sitio web donde muchos usuarios no visitan el sitio.

In [ ]:
# Instalación de paquetes necesarios (si es necesario)
# !pip install statsmodels

# Ajuste del modelo cero-inflado (utilizando ZeroInflatedPoisson de statsmodels)
from statsmodels.discrete.count_model import ZeroInflatedPoisson

model_zero_inflated = ZeroInflatedPoisson(data_poisson['defects'], sm.add_constant(data_poisson['hours'])).fit()

# Resumen del modelo
print(model_zero_inflated.summary())

Interpretación: El modelo incluye un componente para el exceso de ceros, permitiendo una mejor representación de los datos.

## 8. Pruebas Post Hoc y Efectos Marginales <a id='post-hoc'></a>

Las pruebas post hoc y los efectos marginales son útiles para interpretar los resultados de los modelos y realizar comparaciones entre grupos. Utilizamos la librería `statsmodels` para realizar estas estimaciones y comparaciones múltiples.

In [ ]:
# Ajuste de medias marginales utilizando statsmodels
import statsmodels.stats.multicomp as multi

# Ajuste ANOVA y comparaciones post hoc
anova_results = sm.stats.anova_lm(result_normal, typ=2)
print(anova_results)

Interpretación: Estas pruebas nos ayudan a entender mejor las diferencias entre los grupos después del ajuste del modelo.

## 9. Conclusiones <a id='conclusions'></a>

En este documento, hemos explorado cómo aplicar modelos lineales mixtos en Python utilizando diferentes distribuciones de datos. Cada distribución tiene sus propias características y es importante seleccionar la adecuada para modelar correctamente los datos.

Los modelos lineales mixtos nos permiten capturar tanto efectos fijos como aleatorios, proporcionando una comprensión más completa de los datos jerárquicos o con medidas repetidas.

## 10. Referencias <a id='references'></a>

- Pinheiro, J. C., & Bates, D. M. (2000). *Mixed-Effects Models in S and S-PLUS*. Springer.
- Gelman, A., & Hill, J. (2007). *Data Analysis Using Regression and Multilevel/Hierarchical Models*. Cambridge University Press.
- [Statsmodels Documentation](https://www.statsmodels.org/stable/index.html)
- [Pandas Documentation](https://pandas.pydata.org/)
- [Numpy Documentation](https://numpy.org/)